## Part 2a) Assigment

In [7]:
import torch
from torch import nn, Tensor
import numpy as np
import random
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
from torchvision.transforms import ToTensor
from torchvision.io import decode_image
import os
from pathlib import Path
import pandas as pd
import seaborn as sns
from typing import List, Tuple, Dict, Optional

In [8]:

dataset_paths = ["/Users/leonardomassaro/Desktop/DatasetOxford/", "/scratch.hpc/leomass/"]
correct_path = None

for path in dataset_paths:
    if os.path.exists(path) and os.path.isdir(path):
        print("Detected dataset on ", path)
        correct_path = path
if not correct_path:
    raise Exception("No dataset found")

Detected dataset on  /Users/leonardomassaro/Desktop/DatasetOxford/


In [9]:
def fix_random(seed: int) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

fix_random(42)

In [10]:
class OxfordPetDataset(Dataset):
    def __init__(self, path, transform=None) -> None:
        super().__init__()

        self.root = Path(path)
        self.names, self.labels = self._get_names_and_labels()
        self.transform = transform

    def __len__(self) -> int:
        return len(self.labels)

    def __getitem__(self, idx) -> Tuple[Tensor, int]:
        img_path = self.root / "images" / f"{self.names[idx]}.jpg"
        img = decode_image(img_path).to(torch.float)
        if img == None or img.shape[0] != 3:
            print("Bad channel formatting for image: ", img_path.__str__())
            print("Using next item as substitute...")
            return self.__getitem__(idx+1)

        label = self.labels[idx]

        if self.transform:
            img = self.transform(img)

        return img, label

    def get_num_classes(self) -> int:
        return max(self.labels) + 1

    def _get_names_and_labels(self) -> Tuple[List[str], List[int]]:
        names = []
        labels = []

        with open(self.root / "annotations" / "list.txt") as f:
            for line in f:
                if(line[0] == "#"):
                    continue
                name, label, _, _ = line.replace("\n", "").split(" ")
                names.append(name),
                labels.append(int(label) - 1)

        return names, labels

In [ ]:
BATCH_SIZE = 128

transform_stack = transforms.Compose([
    transforms.Resize((256,256))
])

dataset = OxfordPetDataset(correct_path, transform=transform_stack)

train_ds, validation_ds, test_ds = random_split(dataset, [6/10, 2/10, 2/10])

# Create data loaders.
train_dataloader = DataLoader(train_ds, batch_size=BATCH_SIZE)
validation_dataloader = DataLoader(validation_ds, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_ds, batch_size=BATCH_SIZE)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 3, 256, 256])
Shape of y: torch.Size([64]) torch.int64


In [12]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.net_stack = nn.Sequential(
            #initial block, like resnet
            nn.LazyConv2d(32, 8),
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.MaxPool2d(3),

            #convolutional block - many convolutional blocks can be dense blocks properly concatenated
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.LazyConv2d(32, 3),

            #transition block to reduce channel dimensionality (conv has to be 1x1)
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.LazyConv2d(16, 1),
            nn.AdaptiveAvgPool2d((1,1)),

            # final block for classification
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),
            nn.LazyLinear(37)

        )

    def forward(self, x):
        #x = self.flatten(x)
        result = self.net_stack(x)
        return result

model = NeuralNetwork().to(device)
print(model)

Using mps device
NeuralNetwork(
  (net_stack): Sequential(
    (0): LazyConv2d(0, 32, kernel_size=(8, 8), stride=(1, 1))
    (1): LazyBatchNorm2d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (4): LazyBatchNorm2d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): LazyConv2d(0, 32, kernel_size=(3, 3), stride=(1, 1))
    (7): LazyBatchNorm2d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): LazyConv2d(0, 16, kernel_size=(1, 1), stride=(1, 1))
    (10): AdaptiveAvgPool2d(output_size=(1, 1))
    (11): LazyBatchNorm2d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): AdaptiveAvgPool2d(output_size=(1, 1))
    (14): Flatten(start_dim=1, end_dim=-1)
    (15): LazyLinear(in_features=0, out_features=37, bias=True)
  )
)


In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [14]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [15]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 20

import time

t1 = time.time()

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

t2 = time.time()

print("Done in ", t2-t1, " msec")

Epoch 1
-------------------------------
loss: 3.679566  [   64/ 4410]


Corrupt JPEG data: premature end of data segment


Bad channel formatting for image:  /Users/leonardomassaro/Desktop/DatasetOxford/images/Abyssinian_5.jpg
Using next item as substitute...
Bad channel formatting for image:  /Users/leonardomassaro/Desktop/DatasetOxford/images/Egyptian_Mau_14.jpg
Using next item as substitute...


Corrupt JPEG data: 240 extraneous bytes before marker 0xd9


Bad channel formatting for image:  /Users/leonardomassaro/Desktop/DatasetOxford/images/Egyptian_Mau_186.jpg
Using next item as substitute...
Test Error: 
 Accuracy: 2.1%, Avg loss: 3.668482 

Epoch 2
-------------------------------
loss: 3.653667  [   64/ 4410]


...

Bad channel formatting for image:  /Users/leonardomassaro/Desktop/DatasetOxford/images/Abyssinian_5.jpg
Using next item as substitute...
Bad channel formatting for image:  /Users/leonardomassaro/Desktop/DatasetOxford/images/Egyptian_Mau_14.jpg
Using next item as substitute...


Corrupt JPEG data: 240 extraneous bytes before marker 0xd9


Bad channel formatting for image:  /Users/leonardomassaro/Desktop/DatasetOxford/images/Egyptian_Mau_186.jpg
Using next item as substitute...
Test Error: 
 Accuracy: 2.5%, Avg loss: 3.647692 

Epoch 3
-------------------------------
loss: 3.628869  [   64/ 4410]


Corrupt JPEG data: premature end of data segment


KeyboardInterrupt: 